In [51]:
import os
import json
import pandas as pd
import traceback

In [69]:
from langchain_community.llms import HuggingFaceHub

In [70]:
from dotenv import load_dotenv
load_dotenv()
HF_KEY = os.getenv("HUGGINGFACEHUB_API_TOKEN")
os.environ["HUGGINGFACEHUB_API_TOKEN"]=HF_KEY


In [86]:
llm = HuggingFaceHub(
    repo_id="google/flan-t5-large", 
    model_kwargs={
        "temperature": 0.5,
        "max_new_tokens": 1024
    }
)

In [87]:
response = llm.invoke("Generate a multiple choice question about Python programming.")
print(response)

AttributeError: 'InferenceClient' object has no attribute 'post'

In [72]:

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
import PyPDF2

In [73]:
RESPONSE_JSON = json.dumps({
  "1": {
    "mcq": "multiple choice question",
    "options": {
      "a": "choice here",
      "b": "choice here",
      "c": "choice here",
      "d": "choice here"
    },
    "correct": "correct answer"
  },
  "2": {
    "mcq": "multiple choice question",
    "options": {
      "a": "choice here",
      "b": "choice here",
      "c": "choice here",
      "d": "choice here"
    },
    "correct": "correct answer"
  },
  "3": {
    "mcq": "multiple choice question",
    "options": {
      "a": "choice here",
      "b": "choice here",
      "c": "choice here",
      "d": "choice here"
    },
    "correct": "correct answer"
  },
  "4": {
    "mcq": "multiple choice question",
    "options": {
      "a": "choice here",
      "b": "choice here",
      "c": "choice here",
      "d": "choice here"
    },
    "correct": "correct answer"
  },
  "5": {
    "mcq": "multiple choice question",
    "options": {
      "a": "choice here",
      "b": "choice here",
      "c": "choice here",
      "d": "choice here"
    },
    "correct": "correct answer"
  }
})


In [74]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to\
create a quiz of {number} multiple choise questions for {subject} students in {tone}.
make sure the questions are not repeated and check all the quesions to be conforming the text as wll.
make sure to format your response like RESPONSE_JSON bellow and use it as a guide.\
Ensure to make {number} of MCQs.
### RESPONSE_JSON
{response_json}
"""

In [75]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone" , "response_json"],
    template=TEMPLATE
)

In [76]:
quiz_chain = LLMChain(llm = llm, prompt = quiz_generation_prompt, output_key = 'quiz', verbose=True)

In [77]:
TEMPLATE2 = """
You are an expert english grammarian and writer.Given a multiple choice quiz for a {subject} students.\
We need to evaluate the complexity of the questions and give a complete analysis of the quiz.Only use at max 50 words for complaxity if the quiz is at per the with the cognitive and analytical ability of the students.\
Update the quiz questions which needs to be changed and change the tone such that it perfectly fits the students abilities.
Quiz_MCQs:
{quiz}

check from an expert english writer of the above quiz:
"""

In [78]:
quiz_evaluation_prompt = PromptTemplate(input_variables=["subject", "quiz"],template=TEMPLATE2)

In [79]:
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt,output_key="review", verbose=True)

In [80]:
generate_evaluation_chain = SequentialChain(chains = [quiz_chain,review_chain], input_variables = ["text", "number", "subject", "tone" , "response_json"], output_variables = ["quiz","review"], verbose = True)

In [81]:
file_path = r"D:\DirectEd\Geni AI\MCQgenerator\data.txt"

In [82]:
with open(file_path, "r") as file:
    TEXT = file.read()
 

In [83]:
NUMBER = 5
SUBJECT = "Machine Learning"
TONE = "Simple"

In [84]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

# How to setup token usage tracking in langchain
response = generate_evaluation_chain.invoke(
    {
        "text":TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "response_json": json.dumps(RESPONSE_JSON)
    }
)

print(response["quiz"])
print(response["review"])

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:

Text:Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks without explicit instructions.[1] Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] The application of ML to business problems is known as predictive analytics.

Statistics and mathematical optimisation (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.[6][7]

From a theoretica

AttributeError: 'InferenceClient' object has no attribute 'post'

In [1]:
quiz = response.get("quiz")

NameError: name 'response' is not defined

In [2]:
quiz_table_data = []

for key, value in quiz.item():
    mcq = value["mcq"]
    options = " | ".join(
        f"{option}:{option_value}"
        for option, option_value in value["options"].items()
    )

    correct = value["correct"]

    quiz_table_data.append(mcq=mcq, options=options, correct=correct)

NameError: name 'quiz' is not defined

In [3]:
quiz = pd.DataFrame(quiz_table_data)

NameError: name 'pd' is not defined

In [4]:
quiz.to_csv("Machinelearningquiz.csv", index=False)

NameError: name 'quiz' is not defined